In [32]:
# imports

import pandas as pd
import os
import requests
import time


### DFCI_2014_PES study

In [33]:
# get data
#TODO import as objects from harvester
## add parameter to skip first 4 lines in patient and study sample data

init_mut_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_mutations.txt', sep='\t')
init_study_meta = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/meta_study.txt', sep='\t')
init_patient_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_clinical_patient.txt', sep='\t', skiprows=4)
init_sample_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_clinical_sample.txt', sep='\t', skiprows=4)


In [34]:
# clean variant data

# subset for necessary columns
mut_df = init_mut_df.filter(['Hugo_Symbol',
                        'Chromosome',
                        'Start_Position', 
                        'End_Position',
                        'Consequence',
                        'Variant_Classification',
                        'Variant_Type',
                        'Reference_Allele',
                        'Tumor_Seq_Allele2',
                        'Tumor_Sample_Barcode',
                        'Sequence_Source',
                        'HGVSc',
                        'HGVSp',
                        'HGVSp_Short',
                        'Transcript_ID',
                        'RefSeq',
                        'Protein_position'
                       ], axis=1)


# Strip whitespace and retry
mut_df.columns = mut_df.columns.str.strip()
mut_df = mut_df.rename(columns={'Tumor_Sample_Barcode': 'SAMPLE_ID'})

# Check duplicate count
num_duplicates = mut_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# TODO check duplicates for sanity check
# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(mut_df[mut_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# save duplicate rows to file
    dupes = mut_df[mut_df.duplicated(keep=False)]
    mut_df.to_csv('mut_dupes.csv', index=False)
    

    
# remove duplicates, but keep first occurrence
    mut_df = mut_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {mut_df.shape}")
else:
    print("No duplicate rows found.")

    
# print(mut_df.columns)
# # print(mut_df.head())

Number of duplicate rows : 495

Duplicate rows (excluding first instance):
      Hugo_Symbol Chromosome  Start_Position  End_Position  \
720         ABCA1          9       107607765     107607765   
730         ABCA3         16         2338066       2338066   
783         ABCC9         12        21954066      21954066   
813         ACACA         17        35640173      35640173   
829          ACHE          7       100490251     100490251   
...           ...        ...             ...           ...   
15134       STAG2          X       123179197     123179197   
15155     ZDHHC15          X        74742823      74742824   
15191       HUWE1          X        53579734      53579734   
15216     SHROOM4          X        50350700      50350700   
15218        SOX3          X       139586714     139586714   

              Consequence Variant_Classification Variant_Type  \
720      missense_variant      Missense_Mutation          SNP   
730      missense_variant      Missense_Mutation  

In [35]:
# clean patient data
# print(patient_df.shape)
# print(patient_df.columns)

# subset data
# subset for necessary columns
patient_df = init_patient_df.filter(['PATIENT_ID',
                        'AGE',
                        'SEX', 
                        'ETHNICITY',
                        'Consequence'
                       ], axis=1)

print(patient_df.columns)

# Check duplicate count
num_duplicates = patient_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(patient_df[patient_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    patient_df = patient_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {patient_df.shape}")
else:
    print("No duplicate rows found.")

Index(['PATIENT_ID', 'AGE', 'SEX', 'ETHNICITY'], dtype='object')
Number of duplicate rows : 0
No duplicate rows found.


In [36]:
# clean sample data

# print(sample_df.shape)
# print(sample_df.columns)
# print(sample_df.head())


# subset data
# subset for necessary columns
sample_df = init_sample_df.filter(['PATIENT_ID',
                                     'SAMPLE_ID',
                                     'SAMPLE_CLASS',
                                     'ONCOTREE_CODE',
                                     'CANCER_TYPE',
                                     'CANCER_TYPE_DETAILED',
                                     'TMB_NONSYNONYMOUS'
                                    ], axis=1)


print(sample_df.columns)


# Check duplicate count
num_duplicates = sample_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(sample_df[sample_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    sample_df = sample_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {sample_df.shape}")
else:
    print("No duplicate rows found.")

Index(['PATIENT_ID', 'SAMPLE_ID', 'SAMPLE_CLASS', 'ONCOTREE_CODE',
       'CANCER_TYPE', 'CANCER_TYPE_DETAILED', 'TMB_NONSYNONYMOUS'],
      dtype='object')
Number of duplicate rows : 0
No duplicate rows found.


In [37]:
# combine dataframes
## TODO: redo so that all columns are available and that records with no samples will be there. "left outer join"? will get NAs.

init_combined_df = mut_df.merge(sample_df, on='SAMPLE_ID', how='left')

# print(mut_df.columns)
# print(mut_df.shape)

# print(sample_df.columns)
# print(sample_df.shape)

# print(init_combined_df.columns)
# print(init_combined_df.shape)

#add patient_df
combined_df = init_combined_df.merge(patient_df, on='PATIENT_ID', how='left')

# print(patient_df.columns)
# print(patient_df.shape)

# print(combined_df.columns)
# print(combined_df.shape)


In [38]:

# add column for study id
study_id = init_study_meta.iloc[0, 0]
study_id = study_id.replace('cancer_study_identifier: ', '')
# study_id
combined_df['STUDY_ID'] = study_id


In [39]:


# Check duplicate count
num_duplicates = combined_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(combined_df[combined_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    combined_df = combined_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {combined_df.shape}")
else:
    print("No duplicate rows found.")



Number of duplicate rows : 0
No duplicate rows found.


In [40]:

# remove cell lines

original_shape = combined_df.shape
print(f"Original shape: {original_shape}")

#lines to remove
removed_df = combined_df[combined_df['SAMPLE_CLASS'] == 'Cell line']

# remove cell lines
filtered_df = combined_df[combined_df['SAMPLE_CLASS'] != 'Cell line']

# calculate how many rows were removed
rows_removed = original_shape[0] - filtered_df.shape[0]
print(f"Removed {rows_removed} rows where SAMPLE_CLASS == 'Cell line'")

# print new shape
print(f"New shape: {filtered_df.shape}")

# reassign df
combined_df = filtered_df

removed_df.to_csv('cell_lines_removed.csv', index=False)
removed_df.value_counts("SAMPLE_CLASS")


Original shape: (14737, 27)
Removed 3123 rows where SAMPLE_CLASS == 'Cell line'
New shape: (11614, 27)


SAMPLE_CLASS
Cell line    3123
Name: count, dtype: int64

In [41]:
# combined_df.isna().sum()

# Hugo_Symbol                   0
# Chromosome                    0
# Start_Position                0
# End_Position                  0
# Consequence                 124 - some of the Variant_Classification=Silent have no consequence described
# Variant_Classification        0
# Variant_Type                  0
# Reference_Allele              0
# Tumor_Seq_Allele2             0
# SAMPLE_ID                     0
# Sequence_Source               0
# HGVSc                       136 - some of the Variant_Classification=Silent and all of the Variant_Classification=3'Flank and 5'Flank
# HGVSp                       346 
# Transcript_ID               124 - some of the Variant_Classification=Silent have no Transcript_ID described
# RefSeq                     1338
# Protein_position            334
# Gnomad_Notation               0
# PATIENT_ID                    0
# SAMPLE_CLASS                  0
# ONCOTREE_CODE                 0
# CANCER_TYPE                   0
# CANCER_TYPE_DETAILED          0
# TMB_NONSYNONYMOUS             0
# AGE                        4745 - some ages undisclosed
# SEX                           0
# ETHNICITY                 10900 - many patients' ethnicities undisclosed
# STUDY_ID                      0


# define output folder (will create it if needed)
output_dir = "value_counts_by_column"
os.makedirs(output_dir, exist_ok=True)

# loop through each column
for col in combined_df.columns:
    # build filename
    filename = f"value_counts_{col}.txt"
    filepath = os.path.join(output_dir, filename)
    
    # write counts to file
    with open(filepath, "w") as f:
        f.write(f"Value counts for column: {col}\n\n")
        f.write(combined_df[col].value_counts(dropna=False).to_string())
        f.write("\n")

print(f"✅ Value counts written for {len(combined_df.columns)} columns to folder: {output_dir}")


✅ Value counts written for 27 columns to folder: value_counts_by_column


In [43]:
#filling in NaNs - AGE, ETHNICITY, Consequence
# TODO add consequence with annotation

cols_to_fill = ['Consequence', 'AGE', 'ETHNICITY']
fill_value = "No_Data"

for col in cols_to_fill:
    combined_df[col] = combined_df[col].fillna(fill_value)

combined_df.isna().sum()

Hugo_Symbol                  0
Chromosome                   0
Start_Position               0
End_Position                 0
Consequence                  0
Variant_Classification       0
Variant_Type                 0
Reference_Allele             0
Tumor_Seq_Allele2            0
SAMPLE_ID                    0
Sequence_Source              0
HGVSc                      136
HGVSp                      346
HGVSp_Short                136
Transcript_ID              124
RefSeq                    1338
Protein_position           334
PATIENT_ID                   0
SAMPLE_CLASS                 0
ONCOTREE_CODE                0
CANCER_TYPE                  0
CANCER_TYPE_DETAILED         0
TMB_NONSYNONYMOUS            0
AGE                          0
SEX                          0
ETHNICITY                    0
STUDY_ID                     0
dtype: int64

In [ ]:
# correcting chromosome 23 to chromosome X or Y

# TODO. look at gene and see whether it maps to x or y for males

# female chromosome 23
combined_df.loc[combined_df.SEX == "Female", 'Chromosome'] = "X"

# male samples 

# duplicate each Male 23 variant, set to X or Y Chromosome, then run through VICC variation normalizer, see which tokenizes. 
# cross reference gene back to dataframe. If it is the wrong one, could be that there is an overlapping gene
# find the rows to duplicate
chr23_rows = combined_df[combined_df['Chromosome'] == 23]
# change Chromosome to Y for subset
chr23_rows.loc[chr23_rows.SEX == "Male", 'Chromosome'] = "Y"
# append them to the original DataFrame
combined_df_male_dup = pd.concat([combined_df, chr23_rows], ignore_index=True)
# change orig male chrom23s to X
combined_df_male_dup.loc[combined_df_male_dup.SEX == "Male", 'Chromosome'] = "X"
#check for duplicates
duplicates = combined_df_male_dup[combined_df_male_dup.duplicated()]
print(duplicates.shape)
# reassign dataframe
combined_df = combined_df_male_dup


# construct Gnomad variant ID column
combined_df["Gnomad_Notation"] = combined_df.apply(
    lambda row: f"{row['Chromosome']}-{row['Start_Position']}-{row['Reference_Allele']}-{row['Tumor_Seq_Allele2']}",
    axis=1
)



# run through VICC variant normalizer
# subset male X/Y variants
xy_male_mask = (
        (combined_df['SEX'] == 'Male') &
        (combined_df['Chromosome'].astype(str).isin(['X', 'Y']))
)
xy_male_df = combined_df.loc[xy_male_mask].copy()

# call the normalizer and extract the gene symbol
def fetch_normalized_symbol(gnomad_notation: str,
                            assembly: str = 'GRCh37') -> str | None:
    """
    Query https://normalize.cancervariants.org/variation/<notation>
    and return the gene symbol if the service provides one.
    """
    url = f"https://normalize.cancervariants.org/variation/{gnomad_notation}"
    try:
        r = requests.get(url, params={'assembly': assembly}, timeout=15)
        r.raise_for_status()
        payload = r.json()

        # gene symbol is usually under payload['gene_context']['gene_symbol']
        gene_ctx = payload.get('gene_context')
        if isinstance(gene_ctx, dict):
            return gene_ctx.get('gene_symbol')

    except (requests.RequestException, ValueError) as exc:
        print(f"[warn] normalizer trouble for {gnomad_notation}: {exc}")

    return None

# annotate every row with Normalized_Symbol
norm_syms: list[str | None] = []
for notation in xy_male_df['Gnomad_Notation']:
    norm_syms.append(fetch_normalized_symbol(notation))
    time.sleep(0.2)

xy_male_df['Normalized_Symbol'] = norm_syms

# merge back into the master dataframe
combined_df.loc[xy_male_df.index, 'Normalized_Symbol'] = xy_male_df['Normalized_Symbol']

# --- 4. report mismatches ----------------------------------------------------
mismatch_mask = (
        (combined_df['SEX'] == 'Male') &
        (combined_df['Chromosome'].astype(str).isin(['X', 'Y'])) &
        (combined_df['Hugo_Symbol'] != combined_df['Normalized_Symbol'])
)

if mismatch_mask.any():
    cols_to_show = ['Hugo_Symbol', 'Normalized_Symbol',
                    'SAMPLE_ID', 'Gnomad_Notation']
    mismatches = combined_df.loc[mismatch_mask, cols_to_show]

    print("\n⚠️  Hugo_Symbol ≠ Normalized_Symbol for the following rows:")
    # DataFrame index = “coordinates”
    print(mismatches.to_string())
else:
    print("✅ No mismatches between Hugo_Symbol and Normalized_Symbol "
          "for male X/Y variants.")







# write to file
combined_df.to_csv('output2.csv', index=False)

(0, 27)
[warn] normalizer trouble for X-159603535-G-C: 403 Client Error: Forbidden for url: https://normalize.cancervariants.org/variation/X-159603535-G-C?assembly=GRCh37
[warn] normalizer trouble for X-35227334-G-A: 403 Client Error: Forbidden for url: https://normalize.cancervariants.org/variation/X-35227334-G-A?assembly=GRCh37
[warn] normalizer trouble for X-1272679-G-A: 403 Client Error: Forbidden for url: https://normalize.cancervariants.org/variation/X-1272679-G-A?assembly=GRCh37
[warn] normalizer trouble for X-140307515-G-A: 403 Client Error: Forbidden for url: https://normalize.cancervariants.org/variation/X-140307515-G-A?assembly=GRCh37
[warn] normalizer trouble for X-102080254-C-T: 403 Client Error: Forbidden for url: https://normalize.cancervariants.org/variation/X-102080254-C-T?assembly=GRCh37
[warn] normalizer trouble for X-133732205-T-C: 403 Client Error: Forbidden for url: https://normalize.cancervariants.org/variation/X-133732205-T-C?assembly=GRCh37
[warn] normalizer tr

KeyboardInterrupt: 

In [48]:
# remove variant dupes per patient

# find duplicated (PATIENT_ID, Gnomad_Notation) pairs
dupe_mask = combined_df.duplicated(subset=["PATIENT_ID", "Gnomad_Notation"], keep="first")
# new DataFrame with the duplicated rows
patient_variant_dupes = combined_df[dupe_mask]
# remove those rows from the original DataFrame
combined_df_cleaned = combined_df[~dupe_mask]
# write removed rows to file
patient_variant_dupes.to_csv("patient_variant_dupes.csv", index=False)
# print the number of rows removed
print(f"Removed {patient_variant_dupes.shape[0]} rows with duplicated Gnomad_Notation per PATIENT_ID.")
# reassign dataframe:
combined_df = combined_df_cleaned

Removed 136 rows with duplicated Gnomad_Notation per PATIENT_ID.
